In [1]:
import psycopg2 
import pandas as pd

In [156]:
def tet_query(query, alias):
    """
    connect to db and save query result to pandas dataframe with alias as columns names
    ----------
    return : pandas dataframe
    """
    conn = psycopg2.connect(user='interview', 
                            password='Abf#59sjgnaaf',
                            host='interview.chmuafipxfh7.eu-central-1.rds.amazonaws.com',
                            port='5432',
                            dbname='interview'
                           )
    cur = conn.cursor()
    cur.execute(query)
    temp_df = pd.DataFrame(cur.fetchall(), columns=alias)
    cur.close()
    conn.close()
    return temp_df

## Аналитика

### Задачи

Продакт-менеджер Василий попросил вас проанализировать завершенные уроки и ответить на следующие вопросы:
1. Какое число наших пользователей активно преподаёт или учится (в разрезе по роли)? Как меняются эти показатели из месяца в месяц?

In [23]:
query = """
        select 
            count(distinct(pupil_id)),
            count(distinct(tutor_id))
        from tet.lessons
        where state = 'finished'
        """
alias = ('Активные ученики', 'Активные преподаватели')
tet_query(query, alias)

,Активные ученики,Активные преподаватели
0,5713,601


In [126]:
query = """
        select 
            to_char((scheduled_time at time zone 'utc' at time zone 'msk'), 
                    'YYYY-MM') as month,
            count(distinct(pupil_id)),
            count(distinct(tutor_id))
        from tet.lessons
        where state = 'finished'
        group by month
        """
alias = ('Месяц', 'Активные ученики', 'Активные преподаватели')
tet_query(query, alias)

,Месяц,Активные ученики,Активные преподаватели
0,2020-08,1501,242
1,2020-09,2714,416
2,2020-10,3756,548


MAU (Monthly Active Users) - одна из важных продуктовых метрик, которая поможет Василию в принятии решений и в оценке событий. Собственно она и показана выше в разрезе по ролям.

---------------------------------------
2. Сколько у нас преподавателей и учеников, которые не участвовали ещё ни в одном уроке? (прогул и отмена считается как участие)

In [44]:
query = """
        select
            role,
            count(id)
        from tet.users as l
        left join 
            (
            select pupil_id 
            from tet.lessons
            ) as r
        on l.id = r.pupil_id
        left join 
            (
            select tutor_id 
            from tet.lessons
            ) as rr
        on l.id = rr.tutor_id
        where 
            r.pupil_id is null
            and
            rr.tutor_id is null
        group by role
        """
alias = ('Роль', 'Количество')
tet_query(query, alias)

,Роль,Количество
0,pupil,1777
1,tutor,314


---------------
3. Сколько у каждого из преподавателя в среднем занятий в неделю (по месяцам)?

In [127]:
query = """
        select
            to_char(week, 'YYYY-MM') as month,
            round(avg(weekly_lessons), 1)
        from
            (
            select
                date_trunc('week', 
                           (scheduled_time at time zone 'utc' at time zone 'msk')
                          )::date +5 as week,
                tutor_id,
                count(id) as weekly_lessons
            from tet.lessons
            where state != 'cancelled'
            group by 
                week,
                tutor_id
            ) as weekly
        group by month
        """
# отмененные уроки не берутся в учет, так как могут искажать реальные показатели
# 
# конкретно на этих данных лучше всего сработает агрегация днец недели к субботе, 
# чтобы не задевать другие месяца
alias = ('Месяц', 'Занятий еженедельно')
tet_query(query, alias)

,Месяц,Занятий еженедельно
0,2020-08,6.2
1,2020-09,7.1
2,2020-10,7.0


--------
4. Сколько уроков по каждому из предметов в месяц не состоялись по причине прогула учителя/ученика (по отдельности каждого + общее кол-во)?

In [128]:
query = """
        select
            l.month,
            l.subject,
            coalesce(pupil_absent, 0),
            coalesce(tutor_absent, 0),
            coalesce(pupil_absent, 0) + coalesce(tutor_absent, 0) as total
        from 
            (
            select 
                to_char((scheduled_time at time zone 'utc' at time zone 'msk'), 
                        'YYYY-MM') as month,
                subject
            from tet.lessons
            group by
                month,
                subject
            ) as l
        left join
            (
            select
                to_char((scheduled_time at time zone 'utc' at time zone 'msk'), 
                        'YYYY-MM') as month,
                subject,
                count(state) as pupil_absent
            from tet.lessons
            where state = 'pupil_no_show'
            group by 
                month,
                subject
            ) as pupils
        on 
            pupils.month = l.month
            and
            pupils.subject = l.subject
        left join
            (
            select
                to_char((scheduled_time at time zone 'utc' at time zone 'msk'), 
                        'YYYY-MM') as month,
                subject,
                count(state) as tutor_absent
            from tet.lessons
            where state = 'tutor_no_show'
            group by 
                month,
                subject
            ) as tutors
        on 
            l.month = tutors.month
            and
            l.subject = tutors.subject
        order by  subject, month
        """
alias = ('Месяц', 'Предмет', 'Прогулов ученика', 'Прогулов учителя', 'Всего прогулов')
tet_query(query, alias)

,Месяц,Предмет,Прогулов ученика,Прогулов учителя,Всего прогулов
0,2020-08,bio,36,8,44
1,2020-09,bio,87,7,94
2,2020-10,bio,83,16,99
3,2020-08,chem,45,6,51
4,2020-09,chem,84,8,92
...,...,...,...,...,...
56,2020-09,speech_therapist,2,0,2
57,2020-10,speech_therapist,51,8,59
58,2020-08,world_around,9,0,9
59,2020-09,world_around,14,0,14


------
5. По каждому предмету определить день недели, в который по этому предмету больше всего уроков + вывести это количество уроков.

In [129]:
query = """
        select
            subject,
            array_agg(weekday),
            lesson_number
        from
            ( 
            select 
                subject,
                to_char((scheduled_time at time zone 'utc' at time zone 'msk'), 
                        'Day') as weekday,
                count(id) as lesson_number,
                max(count(id)) over (partition by subject) as max_lesson
            from tet.lessons
            group by 
                subject,
                weekday
            ) as sub
        where lesson_number = max_lesson
        group by 
            subject,
            lesson_number
        """
alias = ('Предмет', 'День недели', 'Количество уроков')
tet_query(query, alias)

,Предмет,День недели,Количество уроков
0,bio,[Monday ],231
1,chem,[Wednesday],244
2,chin,[Sunday ],7
3,deut,[Tuesday ],52
4,en,[Thursday ],966
5,fre,[Friday ],15
6,geo,[Thursday ],27
7,hist,[Tuesday ],115
8,it,[Friday ],150
9,jap,[Saturday ],13


-----
6. Если в ноябре прирост составит 300 учеников, сколько нам нужно нанять преподавателей, чтобы сохранить на прежнем уровне (как в октябре) утилизацию преподавателей?

Для оценки уровня утилизации преподавателей посмотрим на их отток в октябре.

In [279]:
# Выгрузим всех учеников и преподавателей, участвоваших в уроках и сохраним их в датафрейм 
query = """
        select 
            scheduled_time at time zone 'utc' at time zone 'msk',
            id,
            pupil_id,
            tutor_id
        from tet.lessons
        """
alias = ( 'lesson_time', 'id','pupil_id', 'tutor_id')
users = tet_query(query, alias)
# определим количество активных пользователей с учетом отмененных и пропущенных уроков 
# active_users = users.grou
# Заведем переменную, которая показывает количество дней с последнего урока, на основе которой определим показатель ухода
# tutors_churn['days_from_lesson'] = tutors.lesson_time.max() - tutors.lesson_time
# tutors['days_from_lesson'] = tutors['days_from_lesson'].dt.days

In [280]:
# определим количество активных пользователей с учетом отмененных и пропущенных уроков 
active_users = users.set_index('lesson_time').resample('MS').agg({'pupil_id': 'nunique', 
                                         'tutor_id': 'nunique', 
                                         'id': 'count'}
                                        ).rename(columns={'pupil_id': 'pupil', 
                                                          'tutor_id': 'tutor', 
                                                          'id': 'lessons'})
active_users

,pupil,tutor,lessons
lesson_time,,,
2020-08-01,2070,257,6168
2020-09-01,4036,453,10626
2020-10-01,5393,575,16268


In [281]:
# Определим среднее количество уроков для одного ученика и одного преподавателя
active_users['lessons_per_pupil'] = round(active_users.lessons / active_users.pupil, 2)
active_users['lessons_per_tutor'] = round(active_users.lessons / active_users.tutor, 2)
active_users

,pupil,tutor,lessons,lessons_per_pupil,lessons_per_tutor
lesson_time,,,,,
2020-08-01,2070,257,6168,2.98,24.00
2020-09-01,4036,453,10626,2.63,23.46
2020-10-01,5393,575,16268,3.02,28.29


In [282]:
# Посчитаем отток преподавателей за два предыдущие месяцы. Для этого посмотрим на дату последнего урока у каждого
tutors_churn = users.groupby('tutor_id', as_index=False) \
                    .agg({'lesson_time': 'max'})
# Увеличим дату на месяц, для дальнейшего сравнения с количеством активных преподавателей
tutors_churn['lesson_time'] = tutors_churn['lesson_time'] + pd.DateOffset(months=1)
tutors_churn = tutors_churn.set_index('lesson_time') \
                            .resample('MS') \
                            .tutor_id.count() \
                            .to_frame() \
                            .query('lesson_time < "2020-11-01"') \
                            .rename(columns={'tutor_id': 'churned_tutors'})
tutors_churn

,churned_tutors
lesson_time,
2020-09-01,20
2020-10-01,44


In [283]:
# Посчитаем процент ухода преподавателей
active_users = active_users.merge(tutors_churn, how='left', on='lesson_time').fillna(0)
active_users['churn_rate'] = active_users.churned_tutors / active_users.tutor
active_users

,pupil,tutor,lessons,lessons_per_pupil,lessons_per_tutor,churned_tutors,churn_rate
lesson_time,,,,,,,
2020-08-01,2070,257,6168,2.98,24.00,0.0,0.000000
2020-09-01,4036,453,10626,2.63,23.46,20.0,0.044150
2020-10-01,5393,575,16268,3.02,28.29,44.0,0.076522


In [284]:
# Подготовим предполагаемые данные следующего месяца с зависимостями и костылями
next_month = active_users.loc['2020-10-01'].copy()
next_month.pupil = next_month.pupil + 300
next_month.lessons = round(next_month.pupil * next_month.lessons_per_pupil)
next_month.tutor = round(next_month.lessons / next_month.lessons_per_tutor)
next_month.churned_tutors = round(next_month.tutor * next_month.churn_rate)
next_month.name = next_month.name + pd.DateOffset(months=1)
next_month

pupil                 5693.000000
tutor                  608.000000
lessons              17193.000000
lessons_per_pupil        3.020000
lessons_per_tutor       28.290000
churned_tutors          47.000000
churn_rate               0.076522
Name: 2020-11-01 00:00:00, dtype: float64

In [285]:
# Добавим данные следующего месяца к общему датафрейму и посчитаем нанятых пользователей с учетом оттока для каждого месяца
active_users = active_users.append(next_month)
active_users['new_tutors'] = active_users.tutor - active_users.tutor.shift(1) + active_users.churned_tutors
active_users

,pupil,tutor,lessons,lessons_per_pupil,lessons_per_tutor,churned_tutors,churn_rate,new_tutors
lesson_time,,,,,,,,
2020-08-01,2070.0,257.0,6168.0,2.98,24.00,0.0,0.000000,NaN
2020-09-01,4036.0,453.0,10626.0,2.63,23.46,20.0,0.044150,216.0
2020-10-01,5393.0,575.0,16268.0,3.02,28.29,44.0,0.076522,166.0
2020-11-01,5693.0,608.0,17193.0,3.02,28.29,47.0,0.076522,80.0


##### Для сохранения показателей октября и предполагаемом притоке 300 учеников необходимо нанять еще 80 преподавателей